# Lecture 9 Excelの自動化

## ダッシュボードを作る

In [ ]:
# 圧縮ファイルをColab内に展開
import requests
from io import BytesIO
from zipfile import ZipFile

r = requests.get('https://github.com/shibats/tdl_python_basic/raw/main/Lecture09/ch04.zip')
zip_buffer = BytesIO(r.content)

with ZipFile(zip_buffer) as myzip:
    myzip.extractall()

In [45]:
# エクセルをPythonで扱うための
# ライブラリをインポート
import openpyxl

In [72]:
# Dashboard用のExcelファイルを読み込む
wb = openpyxl.load_workbook("ch04/dashboard.xlsx")
# シートのオブジェクトを得る
ws = wb["Sheet1"]

In [47]:
# pandasに読み込んで表示
import pandas as pd
df = pd.DataFrame(ws.values)
df  # 表示

,0,1,2,3,4,5
0,Aさん,Bさん,Cさん,Dさん,合計,2021-04-21 00:00:00
1,30,30,20,40,=SUM(A2:D2),=E3/E2
2,26,25,11,36,=SUM(A3:D3),=E2-E3
3,24,25,11,32,=SUM(A4:D4),None
4,20,24,11,28,=SUM(A5:D5),None
5,20,23,10,24,=SUM(A6:D6),None
6,None,None,None,None,None,None
7,ひとこと,ひとこと,ひとこと,ひとこと,None,None


In [10]:
# Aさん用のファイルを読み込む
wb_a = openpyxl.load_workbook("ch04/A.xlsx")
# シートのオブジェクトを得る
ws_a = wb_a["Sheet1"]
# pandasに変換して表示
pd.DataFrame(ws_a.values)

,0,1,2,3,4,5,6
0,Aさん用シート,None,None,None,None,None,None
1,30,26,24,20,20,None,もう少し！
2,目標,当日の成果,前日の成果,2日前の成果,3日前の成果,None,ひとこと


In [27]:
# ループでA〜Dさんのファイルを取得
pds = []
for name in ["A", "B", "C", "D"]:
    filename = f"ch04/{name}.xlsx"
    wb_tmp = openpyxl.load_workbook(filename)
    ws_tmp = wb_tmp["Sheet1"]
    pds.append(pd.DataFrame(ws_tmp.values))

In [28]:
pds[0]  # Aさん

,0,1,2,3,4,5,6
0,Aさん用シート,None,None,None,None,None,None
1,30,26,24,20,20,None,もう少し！
2,目標,当日の成果,前日の成果,2日前の成果,3日前の成果,None,ひとこと


In [29]:
pds[1]  # Bさん

,0,1,2,3,4,5,6
0,Bさん用シート,None,None,None,None,None,None
1,30,26,25,25,24,None,がんばろう
2,目標,当日の成果,前日の成果,2日前の成果,3日前の成果,None,ひとこと


In [30]:
pds[2]  # Cさん

,0,1,2,3,4,5,6
0,Cさん用シート,None,None,None,None,None,None
1,20,11,11,11,11,None,壁が高い
2,目標,当日の成果,前日の成果,2日前の成果,3日前の成果,None,ひとこと


In [32]:
pds[3]  # Dさん

,0,1,2,3,4,5,6
0,Dさん用シート,None,None,None,None,None,None
1,40,38,36,32,28,None,手伝いますよ！
2,目標,当日の成果,前日の成果,2日前の成果,3日前の成果,None,ひとこと


In [73]:
# 更新前
pd.DataFrame(ws.values)

,0,1,2,3,4,5
0,Aさん,Bさん,Cさん,Dさん,合計,2021-04-21 00:00:00
1,30,30,20,40,=SUM(A2:D2),=E3/E2
2,26,25,11,36,=SUM(A3:D3),=E2-E3
3,24,25,11,32,=SUM(A4:D4),None
4,20,24,11,28,=SUM(A5:D5),None
5,20,23,10,24,=SUM(A6:D6),None
6,None,None,None,None,None,None
7,ひとこと,ひとこと,ひとこと,ひとこと,None,None


In [80]:
# Dashboard用のExcelファイルを読み込む
wb = openpyxl.load_workbook("ch04/dashboard.xlsx")
# シートのオブジェクトを得る
ws = wb["Sheet1"]

# 各シートの内容をダッシュボードに転記
for idx, name in enumerate(["A", "B", "C", "D"]):
    # 個人用シートを得る
    filename = f"ch04/{name}.xlsx"
    wb_tmp = openpyxl.load_workbook(filename)
    ws_tmp = wb_tmp["Sheet1"]
    # 直近4日の達成数を転記
    for idx2, cell in enumerate(ws_tmp[f"B2:E2"][0]):
        # ダッシュボードの数値を更新
        ws[name][idx2+2].value = cell.value
    ws[name][7].value = ws_tmp["G2"].value

In [81]:
# 更新後
pd.DataFrame(ws.values)

,0,1,2,3,4,5
0,Aさん,Bさん,Cさん,Dさん,合計,2021-04-21 00:00:00
1,30,30,20,40,=SUM(A2:D2),=E3/E2
2,28,26,11,38,=SUM(A3:D3),=E2-E3
3,26,25,11,36,=SUM(A4:D4),None
4,24,25,11,32,=SUM(A5:D5),None
5,20,24,11,28,=SUM(A6:D6),None
6,None,None,None,None,None,None
7,もう少し！,がんばろう,壁が高い,手伝いますよ！,None,None


In [82]:
# 保存
wb.save("ch04/dashboard_new.xlsx")